In [10]:
!pip install graphdatascience

In [11]:
from graphdatascience import GraphDataScience

In [12]:
import numpy as np

In [13]:
NEO4J_URI= 'bolt://localhost:7687'
NEO4J_USER = "neo4j"
NEO4J_PASSWORD ="password"

In [14]:
# Use Neo4j URI and credentials according to your setup
# NEO4J_URI could look similar to "bolt://my-server.neo4j.io:7687"
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
# print(gds.version())

# assert gds.version()

In [15]:
from neo4j import GraphDatabase
uri = NEO4J_URI
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))


In [16]:
gds.run_cypher("""
WITH
  point({latitude: -32.01019274128029,longitude:115.88821652595887, crs: 'WGS-84'}) AS p1,
  point({latitude: -32.0102316121809, longitude:115.88907222154214, crs: 'WGS-84'}) AS p2
RETURN point.distance(p1,p2) AS dist
""")

,dist
0,80.888188


In [17]:
import osmnx as ox

In [27]:
G = ox.load_graphml('perth-drive-con-unprojected-risk.graphml')

In [28]:
gdf_nodes, gdf_relationships = ox.graph_to_gdfs(G)

In [29]:
gdf_nodes.reset_index(inplace=True)
gdf_relationships.reset_index(inplace=True)

## Create a constraint

In [30]:
gds.run_cypher("""
CREATE CONSTRAINT IF NOT EXISTS  FOR (i:Intersection) REQUIRE i.osmid IS UNIQUE
""")

""


In [31]:
gds.run_cypher("""
CREATE INDEX IF NOT EXISTS FOR ()-[r:ROAD_SEGMENT]-() ON r.osmid
""")

""


In [32]:
node_query = '''
    UNWIND $rows AS row
    WITH row WHERE row.osmid IS NOT NULL
    MERGE (i:Intersection {osmid: row.osmid})
        SET i.location =
         point({latitude: row.y, longitude: row.x }),
            i.ref = row.ref,
            i.highway = row.highway,
            i.street_count = toInteger(row.street_count),
            i.hasEvent = row.hasEvent,
            i.isBlocked = row.isBlocked,
            i.risk_lvl = row.risk_lvl
    RETURN COUNT(*) as total
    '''

rels_query = '''
    UNWIND $rows AS road
    MATCH (u:Intersection {osmid: road.u})
    MATCH (v:Intersection {osmid: road.v})
    MERGE (u)-[r:ROAD_SEGMENT {osmid: road.osmid}]->(v)
        SET r.oneway = road.oneway,
            r.lanes = road.lanes,
            r.ref = road.ref,
            r.name = road.name,
            r.highway = road.highway,
            r.max_speed = road.maxspeed,
            r.length = toFloat(road.length)
    RETURN COUNT(*) AS total
    '''

def insert_data(tx, query, rows, batch_size=1000):
    total = 0
    batch = 0

    while batch * batch_size < len(rows):
        results = tx.run(query, parameters = {
          'rows':
            rows[batch * batch_size: (batch + 1) * batch_size]
            .to_dict('records')
        }).data()
#         print(results)
        total += results[0]['total']
        batch += 1

with driver.session() as session:
    session.execute_write(insert_data, node_query, gdf_nodes.drop(columns=['geometry']))
    session.execute_write(insert_data, rels_query, gdf_relationships.drop(columns=['geometry']))

Transaction failed and will be retried in 1.1871782510096578s (The allocation of an extra 2.0 MiB would use more than the limit 716.8 MiB. Currently using 714.9 MiB. dbms.memory.transaction.total.max threshold reached)


TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 2.0 MiB would use more than the limit 716.8 MiB. Currently using 714.9 MiB. dbms.memory.transaction.total.max threshold reached}